In [1]:
import pandas as pd
import numpy as np

In [2]:
#!pip install tensorflow-datasets

In [3]:
columns_map = {
    'Doc. Date': 'PedidoDataDocumento',
    'Material': 'PedidoProdutoId',
    'Order qty': 'PedidoOrdem',
    'Ship-to nu': 'PedidoAtacadistaId',
    'PCS delivered': 'PedidoQuantidade',
    'HL delivered': 'PedidoHectolitroEntregue',
    'delivery_days': 'PedidoDiasEntrega',
    'delivery_flag': 'PedidoEntregue',
    'lead_time_creation_vs_rdd_flag': 'PedidoLeadTimeEntrega',
    'MACO/HL ': 'PedidoContribuicaoHectolitro',
    'Groupement': 'AtacadistaGrupoImpostoPreco',
    'Postal Code': 'AtacadistaCodigoPostal',
    'Street': 'AtacadistaRuaEndereco',
    'Sous groupement': 'AtacadistaSubgrupo',
    'M2_Territory_ID': 'AtacadistaLocalizacaoId',
    'M1_Territory_ID': 'AtacadistaSubLocalizacaoId',
    'Dépt': 'AtacadistaDepartamento',
    'Brand': 'ProdutoMarca',
    'Subrand': 'ProdutoSubmarca',
    'SEGMENTS : Pils / Spécialités / Superspécialités/Bouteille Young adult': 'ProdutoSegmento2',
    'Container Type': 'ProdutoTipoContainer',
    'Container Size': 'ProdutoTamanhoContainer',
    'Variétés': 'ProdutoVariedade',
    'Segment LE': 'ProdutoSegmento1',
    'Latitude': 'AtacadistaLatitude',
    'Longitude': 'AtacadistaLongitude',
    'Degre Alc': 'ProdutoPercentAlcoolico',
    'TTC': 'PedidoPrecoFinalParaUmProd',
    'Brut + TE': 'PedidoTaxas',
    'DA': 'PedidoTaxaTransporteAereo',
    'Net + TE - Hors majoration de rompu de palette': 'PedidoPrecoLiquidoDescontoPaliteQuebrado'
}

In [4]:
df = pd.read_excel('Data.xlsx')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.columns = df.columns.map(columns_map)

In [5]:
df_produto = pd.read_csv('produto_model.csv', index_col='Unnamed: 0')
df_atacadista = pd.read_csv('df_atacadista.csv', index_col='Unnamed: 0')
df_pedido = df[[column for column in df.columns if 'Pedido' in column]]

In [6]:
df_produto.head()
df_produto.drop(columns=['cluster'], inplace=True)

In [7]:
df_pedido = df_pedido.reset_index(drop=True)

In [8]:
filtro_produtos = df_pedido['PedidoProdutoId'].isin(df_produto['PedidoProdutoId'])
filtro_atacadistas = df_pedido['PedidoAtacadistaId'].isin(df_atacadista['PedidoAtacadistaId'])
df_pedido = df_pedido[filtro_produtos & filtro_atacadistas]
df_pedido.reset_index(drop=True, inplace=True)
df_pedido

,PedidoDataDocumento,PedidoProdutoId,PedidoOrdem,PedidoAtacadistaId,PedidoQuantidade,PedidoHectolitroEntregue,PedidoDiasEntrega,PedidoEntregue,PedidoLeadTimeEntrega,PedidoContribuicaoHectolitro,PedidoPrecoFinalParaUmProd,PedidoTaxas,PedidoPrecoLiquidoDescontoPaliteQuebrado,PedidoTaxaTransporteAereo
0,2021-01-11,10946,5.0,29606863,350.0,21.0,7,1,1,150.689825,18.137448,15.582,15.114540,0.0
1,2021-01-11,19898,2.0,29606863,140.0,8.4,7,1,1,141.207904,17.396467,14.688,14.497056,0.0
2,2021-01-11,80176,4.0,29606863,280.0,16.8,7,1,1,92.872307,12.279859,10.368,10.233216,0.0
3,2021-01-11,66989,1.0,29606863,70.0,4.2,7,1,1,195.334035,21.996000,18.330,18.330000,0.0
4,2021-01-08,11766,70.0,29378784,70.0,4.2,6,1,1,142.902378,17.359200,14.466,14.466000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73665,2019-12-30,17028,2.0,29446255,140.0,8.4,8,1,1,217.449263,23.575622,20.856,19.646352,0.0
73666,2019-12-30,33335,2.0,29446255,140.0,8.4,8,1,1,167.128531,19.770696,17.490,16.475580,0.0
73667,2019-12-30,53566,2.0,29446255,140.0,8.4,8,1,1,211.631664,23.575622,20.856,19.646352,0.0
73668,2019-12-30,63482,1.0,29446255,70.0,4.2,8,1,1,122.037438,14.576040,12.786,12.146700,0.0


In [9]:
df = df_pedido[['PedidoDataDocumento', 'PedidoProdutoId', 'PedidoAtacadistaId', 'PedidoQuantidade']]

In [10]:
df = pd.merge(df, df_produto, how='inner', left_on='PedidoProdutoId', right_on='PedidoProdutoId')[['PedidoDataDocumento', 'PedidoProdutoId', 'PedidoAtacadistaId', 'PedidoQuantidade', 'cluster_caracteristica']]

In [11]:
df.dropna(inplace=True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73666 entries, 0 to 73669
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   PedidoDataDocumento     73666 non-null  datetime64[ns]
 1   PedidoProdutoId         73666 non-null  int64         
 2   PedidoAtacadistaId      73666 non-null  int64         
 3   PedidoQuantidade        73666 non-null  float64       
 4   cluster_caracteristica  73666 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 3.4 MB


In [13]:
df = pd.merge(df, df_atacadista, how='inner', left_on='PedidoAtacadistaId', right_on='PedidoAtacadistaId')[['PedidoProdutoId', 'PedidoAtacadistaId', 'PedidoQuantidade', 'cluster_caracteristica', 'cluster']]

In [14]:
df = df.rename(columns={'cluster_caracteristica': 'ProdutoCluster', 'cluster': 'AtacadistaCluster'})

In [15]:
df.drop(df.loc[df['PedidoQuantidade'] == 0].index, inplace=True)

In [16]:
df.to_csv('df_recomendacao.csv')

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68364 entries, 0 to 73665
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   PedidoProdutoId     68364 non-null  int64  
 1   PedidoAtacadistaId  68364 non-null  int64  
 2   PedidoQuantidade    68364 non-null  float64
 3   ProdutoCluster      68364 non-null  int64  
 4   AtacadistaCluster   68364 non-null  int64  
dtypes: float64(1), int64(4)
memory usage: 3.1 MB


In [18]:
import tensorflow as tf
import tensorflow_recommenders as tfrs

In [19]:
data = tf.data.Dataset.from_tensor_slices(
    {
        'PedidoProdutoId': tf.cast(df['PedidoProdutoId'], tf.int32),
        'PedidoAtacadistaId': tf.cast(df['PedidoAtacadistaId'], tf.int32),
        'ProdutoCluster': tf.cast(df['ProdutoCluster'], tf.int32),
        'AtacadistaCluster': tf.cast(df['AtacadistaCluster'], tf.int32),
        'PedidoQuantidade': tf.cast(df['PedidoQuantidade'], tf.float32),
    } 
)

In [20]:
tf.random.set_seed(42)
shuffled = data.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [21]:
features_names = ['PedidoProdutoId', 'PedidoAtacadistaId', 'ProdutoCluster', 'AtacadistaCluster']

vocabularies = {}

for feature_name in features_names:
  vocab = data.batch(len(df)).map(lambda x: x[feature_name])
  vocabularies[feature_name] = np.unique(np.concatenate(list(vocab)))

In [22]:
vocabularies

{'PedidoProdutoId': array([   3337,    3338,    3371,    3372,    3410,    3411,    3450,
           3507,    3530,    3643,    5983,    6013,    6108,    9937,
           9938,    9974,    9976,   10611,   10687,   10946,   10947,
          10957,   10958,   10965,   11574,   11766,   15709,   15710,
          15740,   15757,   15793,   15800,   15985,   16519,   16550,
          17028,   18690,   18827,   19898,   20527,   20614,   21416,
          29777,   30822,   33335,   34906,   37961,   39616,   40093,
          43113,   45416,   45918,   46093,   47934,   48537,   48538,
          48859,   50453,   53236,   53425,   53566,   54232,   54632,
          54639,   55357,   57005,   58245,   58277,   58328,   58941,
          59286,   59847,   59848,   59873,   59874,   59895,   59939,
          59940,   59941,   59942,   60036,   60387,   60505,   61629,
          61684,   61685,   61687,   61943,   62033,   62034,   62474,
          62706,   62707,   62770,   63285,   63311,   634

In [23]:
class DCN(tfrs.Model):

    def __init__(self, use_cross_layer, deep_layer_sizes, projection_dim=None):
        super().__init__()

        self.embedding_dimension = 32

        self._all_features = features_names
        self._embeddings = {}

        for feature_name in self._all_features:
            vocabulary = vocabularies[feature_name]
            self._embeddings[feature_name] = tf.keras.Sequential(
                [tf.keras.layers.IntegerLookup(vocabulary=vocabulary, mask_token=None),
                tf.keras.layers.Embedding(len(vocabulary), self.embedding_dimension)])

        if use_cross_layer:
            self._cross_layer = tfrs.layers.dcn.Cross(
                projection_dim=projection_dim,
                kernel_initializer="glorot_uniform")
        else:
            self._cross_layer = None

        self._deep_layers = [tf.keras.layers.Dense(layer_size, activation="relu")
        for layer_size in deep_layer_sizes]

        self._logit_layer = tf.keras.layers.Dense(1)

        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError("RMSE")])

    def call(self, features):
        embeddings = []
        for feature_name in self._all_features:
            embedding_fn = self._embeddings[feature_name]
            embeddings.append(embedding_fn(features[feature_name]))

        x = tf.concat(embeddings, axis=1)

        # Build Cross Network
        if self._cross_layer is not None:
            x = self._cross_layer(x)

        # Build Deep Network
        for deep_layer in self._deep_layers:
            x = deep_layer(x)

        return self._logit_layer(x)

    def compute_loss(self, features, training=False):
        labels = features.pop("PedidoQuantidade")
        scores = self(features)
        return self.task(
            labels=labels,
            predictions=scores)

In [24]:
epochs = 8
learning_rate = 0.01

In [25]:
cached_train = train.shuffle(100_000).batch(4096).cache()
cached_test = test.batch(4096).cache()

In [30]:
def run_models(use_cross_layer, deep_layer_sizes, projection_dim=None, num_runs=5):
  models = []
  rmses = []

  for i in range(num_runs):
    model = DCN(use_cross_layer=use_cross_layer,
                deep_layer_sizes=deep_layer_sizes,
                projection_dim=projection_dim)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate))
    models.append(model)

    history = model.fit(cached_train, epochs=epochs, verbose=False)
    print(history)
    metrics = model.evaluate(cached_test, return_dict=True)
    # rmses.append(metrics["RMSE"])

  mean, stdv = np.average(rmses), np.std(rmses)

  return {"model": models, "mean": mean, "stdv": stdv}

In [31]:
dcn_result = run_models(use_cross_layer=True, deep_layer_sizes=[50, 50])

C:\Users\angel\anaconda3\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\angel\anaconda3\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\angel\anaconda3\lib\site-packages\numpy\core\_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\angel\anaconda3\lib\site-packages\numpy\core\_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
C:\Users\angel\anaconda3\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## Baseado em filtragem

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs

In [ ]:
df = pd.read_csv('df_recomendacao.csv', index_col='Unnamed: 0')
df.head()

In [ ]:
pedidos = tf.data.Dataset.from_tensor_slices(
    {
        'PedidoProdutoId': tf.cast(df['PedidoProdutoId'], tf.int32),
        'PedidoAtacadistaId': tf.cast(df['PedidoAtacadistaId'], tf.int32),
        'AtacadistaCluster': tf.cast(df['AtacadistaCluster'], tf.int32),
        'PedidoQuantidade': tf.cast(df['PedidoQuantidade'], tf.float32),
    } 
)

uniq_produto = df[['PedidoProdutoId', 'ProdutoCluster']].drop_duplicates().reset_index()
produtos = tf.data.Dataset.from_tensor_slices(
    {
        'PedidoProdutoId': tf.cast(uniq_produto['PedidoProdutoId'], tf.int32),
        'ProdutoCluster': tf.cast(uniq_produto['ProdutoCluster'], tf.int32)
    }
)

In [ ]:
tf.random.set_seed(42)
shuffled = pedidos.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [ ]:
produtos_id = produtos.batch(100).map(lambda x: x['PedidoProdutoId'])
atacadistas_id = pedidos.batch(1000).map(lambda x: x['PedidoAtacadistaId'])

In [ ]:
unique_user_ids = np.unique(np.concatenate(list(atacadistas_id)))
unique_produtos_id = np.unique(np.concatenate(list(produtos_id)))

In [ ]:
embedding_dimension = 32

In [ ]:
atacadista_model = tf.keras.Sequential([
  tf.keras.layers.IntegerLookup(
      vocabulary=unique_user_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [ ]:
produto_model = tf.keras.Sequential([
  tf.keras.layers.IntegerLookup(
      vocabulary=unique_produtos_id, mask_token=None),
  tf.keras.layers.Embedding(len(unique_produtos_id) + 1, embedding_dimension)
])

In [ ]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=produtos.batch(128).map(lambda x: x['PedidoProdutoId']).map(produto_model)
)

In [ ]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [ ]:
from typing import Dict, Text

In [ ]:
class RecomendacaoModel(tfrs.Model):

  def __init__(self, atacadista_model, produto_model):
    super().__init__()
    self.produto_model: tf.keras.Model = produto_model
    self.atacadista_model: tf.keras.Model = atacadista_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.atacadista_model(features["PedidoAtacadistaId"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_produto_embedding = self.produto_model(features["PedidoProdutoId"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_produto_embedding)

In [ ]:
model = RecomendacaoModel(atacadista_model, produto_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
cached_train = train.shuffle(100_000).batch(256).cache()
cached_test = test.batch(256).cache()

In [ ]:
model.fit(cached_train, epochs=5)

In [ ]:
index = tfrs.layers.factorized_top_k.BruteForce(model.atacadista_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((produtos.batch(100).map(lambda x: x['PedidoProdutoId']), produtos.batch(100).map(lambda x: x['PedidoProdutoId']).map(model.produto_model)))
)

# Get recommendations.
_, titles = index(tf.constant([29606863]))
print(f"Recommendations for user 29606863: {titles[0, :3]}")